In [1]:
# coding: utf-8

####RUN THIS SCRIPT USING this line in the terminal, changing the directory to your fmriprep directory####:

#$ python extract_confounds.py --fmriprepDir="/data/projects/learning_lemurs/derivatives/fmriprep/"

# TO DO:
# 1. write to subject folders, create if it doesn't exist
# 2. simplify input argument to project name. all paths should be standardized within a project folder
# 3. check for existence of output before overwiting older output. helps with version control on datalad.
# 4. give option to overwrite existing output

In [4]:
import numpy as np
import pandas as pd
import argparse
import os
import re
from os.path import join

In [5]:
# some parameters

bids_root = "/data/perlman/moochie/analysis/CARE/MRI_data_clean"
task_label = 'movie'
space_label = 'MNIPediatricAsym'
derivatives_folder = join(bids_root, 'derivatives/fmriprep')
TR = 0.8

In [7]:
# parser = argparse.ArgumentParser(description='Give me a path to your fmriprep output')
# group = parser.add_mutually_exclusive_group(required=True)

# group.add_argument('--fmriprepDir',default=None, type=str,help="This is the full path to your fmriprep dir")
# args = parser.parse_args()

# fmriprep_path = args.fmriprepDir

print("finding confound files located in %s"%(derivatives_folder))
#make list of confound tsvs
cons=[]
for root, dirs, files in os.walk(derivatives_folder):
    for f in files:
        if f.endswith('-confounds_timeseries.tsv'):
            cons.append(os.path.join(root, f))

finding confound files located in /data/perlman/moochie/analysis/CARE/MRI_data_clean/derivatives/fmriprep


In [ ]:
for f in cons:
    sub=re.search('/func/(.*)_task', f).group(1)
    run=re.search('_run-(.*)_desc', f).group(1)
    task=re.search('_task-(.*)_run',f).group(1)
    derivitive_path=re.search('(.*)fmriprep/sub',f).group(1)


    outfile="%s_task-%s_run-%s_desc-fslConfounds.tsv"%(sub,task,run)


    #read in the confounds, aroma mixing, and aroma confound indexes
    con_regs=pd.read_csv(f,sep='\t')

    other=['csf','white_matter']
    cosine = [col for col in con_regs if col.startswith('cosine')]
    NSS = [col for col in con_regs if col.startswith('non_steady_state')]
    #motion_out=[col for col in con_regs if col.startswith('motion_outlier')]
    aroma_motion=[col for col in con_regs if col.startswith('aroma')]

    filter_col=np.concatenate([cosine,NSS,aroma_motion,other])#here we combine all NSS AROMA motion & the rest

    #This Dataframe will be the full filter matrix
    df_all=con_regs[filter_col]
    #change this dir to where you want your output (this puts it in individual subject folders)
    outdir=derivitive_path+"/confounds/%s/" %(sub)

    if not os.path.exists(outdir):
        os.makedirs(outdir)
    output=outdir+outfile
    print(sub,run,task)

    df_all.to_csv(output,index=False,sep='\t',header=False)